In [1]:
from random import Random
import numpy as np
import pandas as pd
from tva.situation import Situation
from tva.strategies import Strategies
import copy
from tva.schemes import Schemes, VotingScheme
from tva.enums import VotingScheme, HappinessFunc, StrategyType
from tva.voter import Voter

In [2]:
# Create the voting situation
situation = Situation(num_voters=4, num_candidates=4, seed=40)
display(situation)

# Apply the voting scheme to the situation
schemes = Schemes()
voting_scheme = VotingScheme.BORDA
schemes.print_results(situation, verbose=True)



Voter 0: ['C', 'B', 'D', 'A']
Voter 1: ['C', 'B', 'D', 'A']
Voter 2: ['C', 'B', 'A', 'D']
Voter 3: ['A', 'C', 'D', 'B']

Anti plurality: C {'C': 4, 'B': 3, 'D': 3, 'A': 2} 
Two voting: C {'C': 4, 'B': 3, 'A': 1, 'D': 0} 
Borda: C {'C': 11, 'B': 6, 'D': 3, 'A': 4}


In [3]:
from copy import deepcopy
from tva.situation import Situation
from tva.enums import HappinessFunc, VotingScheme, StrategyType
from tva.models.BTVA import BTVA
import random

class ATVA2(BTVA):
    def __init__(self):
        BTVA.__init__(self)


    def analyse_single(self, situation: Situation, happiness_func: HappinessFunc, voting_scheme: VotingScheme, verbose=False) -> dict[int, list[dict[str, float|int]]]:
        analysis = {}
        for voter in situation.voters[:1]:
            voter_index = voter.voter_id
            original_total_happiness, original_individual_happiness= situation.calculate_happiness(situation.voters, happiness_func, voting_scheme) # type: ignore

            strategic_preferences = self.strategy.apply_all_strategies_to_voter(situation,voter_index, voting_scheme, happiness_func, exhaustive_search=True, verbose=verbose)
        
            # Save my average happiness for each preference list
            preferences_happiness = []
            for _, preference_list in strategic_preferences.items():
                # For each set of preferences in the list, create a new situation
                for preferences in preference_list:
                    situation_copy = deepcopy(situation)
                    situation_copy.voters[voter_index].preferences = preferences
                    winner = self.schemes.apply_voting_scheme(voting_scheme, situation_copy.voters)
                    # Check the strategic options of each of my opponents, select the ones that maximize their happiness 
                    # then, calculate my happiness after they have chosen their best strategy
                    all_other_voter_ids = [voter.voter_id for voter in situation_copy.voters if voter.voter_id != voter_index]
                    my_happiness_list = []
                    total_happiness_list = []
                    for enemy_index in all_other_voter_ids:
                        enemy_situation = self.__find_situation_chosen_by_enemy(situation_copy, enemy_index, voting_scheme, happiness_func, verbose)
                        # Calculate my happiness after the enemy has chosen their best strategy
                        strategic_total_happiness, strategic_individual_happiness= enemy_situation.calculate_happiness(situation.voters, happiness_func, voting_scheme) # type: ignore

                        my_happiness = strategic_individual_happiness[voter_index]
                        my_happiness_list.append(my_happiness)
                        total_happiness_list.append(strategic_total_happiness)
                    # Calculate my average happiness over all options that my enemies would choose
                    my_average_happiness = np.mean(my_happiness_list)
                    average_total_happiness = np.mean(total_happiness_list)
                    preferences_happiness.append({
                        "strategy": preferences,
                        "strategic_winner":winner,
                        "strategic_individual_happiness": my_average_happiness,
                        'original_individual_happiness': original_individual_happiness[voter_index],
                        'strategic_total_happiness': average_total_happiness,
                        'original_total_happiness': original_total_happiness})

            analysis[voter_index] = preferences_happiness

        return analysis

    def __find_situation_chosen_by_enemy(self, situation:Situation, enemy_index:int, voting_scheme:VotingScheme, happiness_func:HappinessFunc, verbose:bool=False) -> Situation:
        enemy_strategies = self.strategy.apply_all_strategies_to_voter(situation,enemy_index, voting_scheme, happiness_func, exhaustive_search=True, verbose=verbose)
        
        # Save only the best strategy for each enemy
        best_enemy_situation = situation
        best_enemy_happiness = -1
        # Try all the strategies of the enemy voter
        for _, enemy_preference_list in enemy_strategies.items():
            for enemy_preferences in enemy_preference_list:
                enemy_situation = deepcopy(situation)
                enemy_situation.voters[enemy_index].preferences = enemy_preferences
                # Calculate the happiness of the enemy voter
                enemy_happiness = float(enemy_situation.calculate_individual_happiness(enemy_preferences, happiness_func, voting_scheme)) # type: ignore
                if enemy_happiness > best_enemy_happiness:
                    best_enemy_happiness = enemy_happiness
                    best_enemy_situation = enemy_situation
        return best_enemy_situation
    

# Create the voting situation
for i in range(10):
    situation = Situation(num_voters=4, num_candidates=4, seed=i)
    display(situation)

    atva = ATVA2()
    analysis = atva.analyse_single(situation, HappinessFunc.LINEAR, VotingScheme.BORDA, verbose=True)
    display(analysis)

Voter 0: ['A', 'D', 'C', 'B']
Voter 1: ['A', 'D', 'C', 'B']
Voter 2: ['C', 'D', 'B', 'A']
Voter 3: ['C', 'D', 'B', 'A']

Applying strategy COMPROMISING
None
Applying strategy BURYING
[['A', 'D', 'B', 'C']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
None
Applying strategy BULLET
[[]]
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['C', 'B', 'A', 'D']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['C', 'B', 'A', 'D']]
Applying strategy BULLET
None


{0: [{'strategy': ['A', 'D', 'B', 'C'],
   'strategic_winner': 'D',
   'strategic_individual_happiness': np.float64(0.7777777777777777),
   'original_individual_happiness': 0.3333333333333333,
   'strategic_total_happiness': np.float64(2.222222222222222),
   'original_total_happiness': 2.6666666666666665}]}

Voter 0: ['B', 'A', 'D', 'C']
Voter 1: ['D', 'B', 'A', 'C']
Voter 2: ['A', 'B', 'D', 'C']
Voter 3: ['D', 'A', 'B', 'C']

Applying strategy COMPROMISING
None
Applying strategy BURYING
[['B', 'C', 'D', 'A'], ['B', 'C', 'D', 'A']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['D', 'A', 'C', 'B'], ['D', 'C', 'A', 'B'], ['C', 'D', 'A', 'B'], ['D', 'C', 'A', 'B'], ['D', 'C', 'B', 'A']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
None
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['D', 'A', 'C', 'B'], ['A', 'C', 'B', 'D'], ['D', 'C', 'B', 'A'], ['A', 'C', 'B', 'D']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['D', 'A', 'C', 'B'], ['D', 'C', 'A', 'B'], ['C', 'D', 'A', 'B'], ['D', 'C', 'A', 'B'], ['D', 'C', 'B', 'A']]
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
None
Applying strategy BULLET
None
Applying strategy COMPROMISING
None
Applying strategy BURYING
[['D', 'A', 'C', 'B'

...

Applying strategy COMPROMISING
None
Applying strategy BURYING
None
Applying strategy BULLET
None


{0: []}

Voter 0: ['C', 'A', 'B', 'D']
Voter 1: ['A', 'D', 'B', 'C']
Voter 2: ['C', 'D', 'B', 'A']
Voter 3: ['B', 'C', 'D', 'A']

Applying strategy COMPROMISING
None
Applying strategy BURYING
None
Applying strategy BULLET
None


{0: []}

In [2]:
from tva.models.ATVA3 import ATVA3


atva = ATVA3()
print('Original Situation: ')
situation = Situation(10,5, seed=42, info=0.5)
situation.print_preference_matrix()
situation, _ = atva.monte_carlo_best_preferences(situation, VotingScheme.BORDA)
print('\nAfter Monte Carlo Simulation preferences:\n')
situation.print_preference_matrix()
happiness_func = HappinessFunc.EXP
voting_scheme = VotingScheme.VOTE_FOR_TWO
strategy_type = StrategyType.BURYING
# ranking = Schemes().apply_voting_scheme(voting_scheme, situation.voters)
sol = atva.analyse_single(situation, happiness_func, voting_scheme, strategy_type) # type: ignore
atva.display_strategic_data(sol)

Original Situation: 
Preference Matrix:
        V0    V1    V2    V3    V4    V5    V6    V7    V8    V9
Rank 1  A     ?     E     E     B     ?     B     D     ?     D
Rank 2  ?     E     A     ?     D     ?     ?     C     ?     B
Rank 3  E     ?     D     A     E     ?     ?     E     B     C
Rank 4  C     ?     C     C     ?     ?     ?     ?     D     ?
Rank 5  ?     ?     B     D     ?     ?     ?     B     C     ?

After Monte Carlo Simulation preferences:

Preference Matrix:
        V0    V1    V2    V3    V4    V5    V6    V7    V8    V9
Rank 1  A     B     E     E     B     E     B     D     E     D
Rank 2  B     E     A     B     D     A     E     C     A     B
Rank 3  E     C     D     A     E     C     A     E     B     C
Rank 4  C     A     C     C     C     B     C     A     D     E
Rank 5  D     D     B     D     A     D     D     B     C     A
[['E', 'A', 'C', 'D', 'B'], ['A', 'E', 'C', 'D', 'B'], ['C', 'E', 'B', 'D', 'A'], ['C', 'E', 'D', 'B', 'A'], ['C', 'E', 'D', 'B

In [11]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","C","D"]
situation.voters[1].preferences = ["C","A","B","D"]
situation.voters[2].preferences = ["D","B","C","A"]
situation.voters[3].preferences = ["D","B","A","C"]
display(situation)

current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
print("Borda:", current_winner, scores)

# situation.voters[0].preferences = ["B","C","A","D"]
# current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
# print("Borda:", current_winner, scores)

strategies = Strategies()
a = strategies.bury(situation, 0, VotingScheme.BORDA, HappinessFunc.LINEAR, verbose=False, exhaustive_search=True)
print(a)

Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'A', 'B', 'D']
Voter 2: ['D', 'B', 'C', 'A']
Voter 3: ['D', 'B', 'A', 'C']

Borda: B {'A': 6, 'B': 7, 'C': 5, 'D': 6}
[['A', 'C', 'D', 'B']]


In [15]:
situation = Situation(num_voters=4, num_candidates=4)
situation.voters[0].preferences = ["A","B","C","D"]
situation.voters[1].preferences = ["C","A","B","D"]
situation.voters[2].preferences = ["B","D","C","A"]
situation.voters[3].preferences = ["B","A","D","C"]
display(situation)

current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
print("Borda:", current_winner, scores)

# situation.voters[0].preferences = ["B","C","A","D"]
# current_winner, scores = schemes.apply_voting_scheme(VotingScheme.BORDA, situation.voters, return_scores=True)
# print("Borda:", current_winner, scores)

strategies = Strategies()
a = strategies.bullet_vote(situation, 0, VotingScheme.BORDA, HappinessFunc.LINEAR, exhaustive_search=True)
print(a)

Voter 0: ['A', 'B', 'C', 'D']
Voter 1: ['C', 'A', 'B', 'D']
Voter 2: ['B', 'D', 'C', 'A']
Voter 3: ['B', 'A', 'D', 'C']

Borda: B {'A': 7, 'B': 9, 'C': 5, 'D': 3}
None


In [ ]:
import copy

def get_indexes_to_iterate(preferences:list[str], scores:dict[str,int]):
    # Given a list of preferences and the scores of those preferences, return a list of indexes of candidates to the left of the current winner
    # The current winner is the candidate with the highest score
    # Return the indexes of preferences sorted by score in descending order
    # If the current winner is at index 0, return an empty list
    current_winner = max(scores, key=scores.get) # type: ignore
    current_winner_index = preferences.index(current_winner)

    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    print("Sorted scores:", sorted_scores)
    candidates = [x[0] for x in sorted_scores[1:]]
    print("Candidates:", candidates)

    indexes_to_try = []
    for i in candidates:
        candidate_index = preferences.index(i)
        if candidate_index < current_winner_index:
            indexes_to_try.append(candidate_index)
    return indexes_to_try

def swap(situation:Situation, voter_id:int, candidate1_index:int, candidate2_index:int, verbose=False):
    if verbose:
        print(f"Swapping {situation.voters[voter_id].preferences[candidate1_index]} and {situation.voters[voter_id].preferences[candidate2_index]}")
    situation.voters[voter_id].preferences[candidate1_index], situation.voters[voter_id].preferences[candidate2_index] = situation.voters[voter_id].preferences[candidate2_index], situation.voters[voter_id].preferences[candidate1_index]

def compromise(situation:Situation, voter_index:int, voting_scheme:VotingScheme, happiness_func:Happiness, exhaustive_search=False, verbose=False) -> None | list[str]:
    # Copy the situation
    new_situation = copy.deepcopy(situation)
    original_voter = new_situation.voters[voter_index]
    original_preferences = new_situation.voters[voter_index].preferences
    # If the original winner is the first preference of the voter, return False
    original_winner, scores = schemes.apply_voting_scheme(voting_scheme, new_situation.voters, return_scores=True)
    original_winner_index = original_preferences.index(original_winner) # type: ignore
    original_winner_happiness = original_voter.calculate_happiness(original_winner, happiness_func)

    if verbose:
        print(original_preferences)
        print("Borda:", original_winner, scores)

    if original_winner_index == 0:
        return None
    
    all_winning_preferences = []
    indexes_to_iterate = get_indexes_to_iterate(original_preferences, scores) # type: ignore
    # Loop over all candidates to the left of the original winner sorted by score
    for i in indexes_to_iterate:
        # Move that candidate to the first position
        swap(new_situation, voter_index, i, 0, verbose=verbose)
        # Check if the winner changed and if the voter is happier
        new_winner, scores = schemes.apply_voting_scheme(voting_scheme, new_situation.voters, return_scores=True)
        new_winner_happiness = original_voter.calculate_happiness(new_winner, happiness_func)

        if verbose:
            print(new_situation.voters[voter_index].preferences)
            print("Borda:", new_winner, scores)

        if new_winner != original_winner and new_winner_happiness > original_winner_happiness:
            if exhaustive_search:
                all_winning_preferences.append(new_situation.voters[voter_index].preferences)
                print("Found a winning preference")
            else:
                print("Found a winning preference")
                return new_situation.voters[voter_index].preferences

    if all_winning_preferences == []:
        return None
    return all_winning_preferences

compromise(situation, 0, VotingScheme.BORDA, Happiness.LINEAR, verbose=True, exhaustive_search=True)

['A', 'D', 'C', 'B']
Borda: D {'A': 4, 'D': 10, 'C': 5, 'B': 5}
Sorted scores: [('D', 10), ('C', 5), ('B', 5), ('A', 4)]
Candidates: ['C', 'B', 'A']
Swapping A and A
['A', 'D', 'C', 'B']
Borda: D {'A': 4, 'D': 10, 'C': 5, 'B': 5}


Create 1000 situations and check for what fraction of them a strategy is good

We do experiments for each voting scheme separately.<br>
TODO: We need to save, what type of strategy has won this time?

In [1]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
situation = Situation(num_voters=5, num_candidates=4)
situation

NameError: name 'Strategies' is not defined

In [33]:
strategies = Strategies()
voting_scheme = VotingScheme.BORDA
nr_repetitions = 1000
num_voters = 4

strategy_counter = 0
for i in range(nr_repetitions):
    situation = Situation(num_voters=num_voters, num_candidates=4)
    # Check if at least one voter has a good strategy
    for voter_index in range(num_voters):
        if strategies.is_any_strategy_good(situation, voter_index, voting_scheme):
            strategy_counter += 1
            break
        
print(strategy_counter/nr_repetitions)

0.877


Plurality cannot be affected by strategic voting because that would require a voter to vote for a candidate they prefer less than their favorite candidate.

In [32]:
a = np.array([
[358,393,450, 393, 450, 552, 358, 306, 386, 450],
[386,407,386, 341, 450, 412, 474, 341, 386, 513],
[386,412,358, 390, 450, 420, 306, 306, 513, 390],
[513,386,358, 404, 341, 552, 386, 552, 420, 450],
[513,420,341, 306, 450, 358, 474, 306, 420, 420],
[420,390,341, 390, 474, 306, 404, 412, 404, 552],
[450,306,393, 552, 412, 513, 513, 513, 552, 407],
[412,341,407, 513, 306, 513, 450, 412, 552, 450],
[386,450,474, 552, 341, 341, 306, 404, 474, 474],
[393,393,450, 450, 407, 450, 306, 420, 450, 390],
[404,341,412, 420, 407, 420, 407, 407, 450, 513],
[450,450,341, 412, 404, 341, 420, 341, 390, 306],
[390,358,407, 407, 407, 386, 552, 412, 474, 306],
[474,358,404, 306, 306, 306, 412, 450, 420, 404],
[412,306,450, 552, 386, 390, 450, 407, 407, 552]])
single_mean = a.mean()
means = a.mean(axis=0).round(0)
single_mean.round(0)
means - single_mean

array([  8.66, -33.34, -16.34,  11.66, -15.34,   2.66,   0.66, -15.34,
        32.66,  23.66])